In [ ]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/student-data-2/processed_train_student - processed_train_student.csv")
test_df = pd.read_csv("/kaggle/input/student-data-2/processed_test_student - processed_test_student.csv")

all_uids = sorted(set(train_df["UID"].tolist() + test_df["UID"].tolist()))

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

student_df = pd.read_csv("/kaggle/input/project-answer-data/all_project_answers_clean.csv")
ref_df = pd.read_csv("/kaggle/input/project-answers/project_reference_answers.csv")

embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

ref_answers = {}
for project in ["PROJECT1"]:
    ref_text = ref_df[ref_df["project_name"] == project]["reference_answer"].values[0]
    ref_answers[project] = embed_model.encode(ref_text)

results = []
for uid in all_uids:
    row = {"UID": uid}
    for project in ["PROJECT1"]:
        stu_ans = student_df[(student_df["UID"] == uid) & (student_df["project_name"] == project)]["student_answer"]
        if not stu_ans.empty:
            all_answers = " ".join(stu_ans.values)
            stu_emb = embed_model.encode(all_answers)
            sim = cosine_similarity([stu_emb], [ref_answers[project]])[0][0]
            row[f"{project}_similarity"] = round(sim, 4)
        else:
            row[f"{project}_similarity"] = 0.0
    results.append(row)

df_score = pd.DataFrame(results)
df_score.to_csv("/kaggle/working/project_similarity_scores_alluids.csv", index=False)
